In [1]:
!which pip # expect pip to be inside .../Versions/3.x/bin/pip
!echo
!which python
!echo
!which python3 # make sure python3 is a thing
!echo
!spacy # make sure spacy is a thing

/Library/Frameworks/Python.framework/Versions/3.7/bin/pip

/usr/bin/python

/Library/Frameworks/Python.framework/Versions/3.7/bin/python3

/bin/sh: /Library/Frameworks/Python.framework/Versions/3.7/bin/spacy: Permission denied


In [2]:
!pip install spacy

In [3]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
import numpy as np
import pandas as pd
import os
import spacy
import datetime
from urllib.request import Request, urlopen
from urllib.parse import urlencode
import json
import gzip

SERVICE_URL = 'https://babelnet.io/v5/getSynsetIds'
BABEL_RESPONSE_DIR = "babel_responses"
print("BABEL_RESPONSE_DIR[:2]:\t", os.listdir(BABEL_RESPONSE_DIR)[:2],"\n")

DATA_DIR = "state-of-the-state"
FILENAME = "index.csv"
SPEECH_DIR_NAME = "speeches"
TABLE_PATH = os.path.join(DATA_DIR, FILENAME)
SPEECH_DIR = os.path.join(DATA_DIR, SPEECH_DIR_NAME)
print("DATA_DIR:\t", os.listdir(DATA_DIR))
print("SPEECH_DIR[:2]:\t", os.listdir(SPEECH_DIR)[:2])
print("TABLE_PATH:\t", TABLE_PATH)
print("SPEECH_DIR:\t", SPEECH_DIR)
print()
print(datetime.date.today())
print(datetime.datetime.now())

BABEL_RESPONSE_DIR[:2]:	 ['.DS_Store', 'apple_2019-07-21.gzip'] 

DATA_DIR:	 ['.DS_Store', 'words.csv', 'speeches', 'README.md', 'index.csv']
SPEECH_DIR[:2]:	 ['Minnesota_SOTS.txt', 'Indiana_SOTS.txt']
TABLE_PATH:	 state-of-the-state/index.csv
SPEECH_DIR:	 state-of-the-state/speeches

2019-07-21
2019-07-21 23:36:14.428567


In [5]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [6]:
DEBUG = False

try:
    # make a file called .api_key with your API_KEY from BabelNet
    with open(".api_key") as f: # https://babelnet.org/login
        BABEL_API_KEY = f.read()
        f.close()
except:
    # or just for one time use, insert it here
    BABEL_API_KEY = "<INSERT_YOUR_API_KEY>" # https://babelnet.org/login

In [7]:
# read the csv for info on the data
df = pd.read_csv(TABLE_PATH)
df.head()

,state,governor,party,filename,url
0,Alabama,Kay Ivey,R,Alabama_SOTS.txt,https://governor.alabama.gov/remarks-speeches/...
1,Alaska,Mike Dunleavy,R,Alaska_SOTS.txt,https://gov.alaska.gov/newsroom/2019/01/22/201...
2,Arizona,Doug Ducey,R,Arizona_SOTS.txt,https://azgovernor.gov/governor/news/2019/01/g...
3,Arkansas,Asa Hutchinson,R,Arkansas_SOTS.txt,https://governor.arkansas.gov/news-media/speec...
4,California,Gavin Newsom,D,California_SOTS.txt,https://www.gov.ca.gov/2019/02/12/state-of-the...


In [8]:
# dictionary mapping filenames to text content
speech_map = {}

# read the files
for fname in df['filename'].get_values():
    with open( os.path.join(SPEECH_DIR, fname) ) as f:
        speech_map[fname] = f.read()
        f.close()

In [9]:
# https://spacy.io/api/span#set_extension
# extend Span object with is_stop
from spacy.tokens import Span

# if the length is 1 and the word is a stopword
stopword_checker_for_span = lambda span: (len(span) == 1 and type(span[0]).__name__ == "Token" and span[0].is_stop)
Span.set_extension("is_stop", getter=stopword_checker_for_span)

# test ._.is_stop
assert(nlp("I")[:]._.is_stop == True)
assert(nlp("Hello world")[:]._.is_stop == False)

In [10]:
# https://spacy.io/usage/linguistic-features#noun-chunks

speech_noun_map = {}

for state in speech_map:
    doc = nlp(speech_map[state])

    nouns = []
    for chunk in doc.noun_chunks:
        if (chunk._.is_stop): # e.g. if chunk == ["it"]
            if DEBUG: 
                print("stopword?", chunk[0], chunk.text)
            continue

        if (chunk[0].is_stop):
            if len(chunk) == 1:
                continue
            else:
                print("\t...found weird chunk {{", chunk, "}} ", end="")
                chunk = chunk[1:]
                print(f"...fixed... [[ {chunk} ]]")       

        nouns.append(chunk.text)

    print(f"got noun chunks for {state}")
    speech_noun_map[state] = nouns

	...found weird chunk {{ the Alabama Legislature }} ...fixed... [[ Alabama Legislature ]]
	...found weird chunk {{ the Alabama Supreme Court }} ...fixed... [[ Alabama Supreme Court ]]
	...found weird chunk {{ my fellow Alabamians }} ...fixed... [[ fellow Alabamians ]]
	...found weird chunk {{ the form }} ...fixed... [[ form ]]
	...found weird chunk {{ our state }} ...fixed... [[ state ]]
	...found weird chunk {{ At least 23 innocent lives }} ...fixed... [[ least 23 innocent lives ]]
	...found weird chunk {{ the good Lord }} ...fixed... [[ good Lord ]]
	...found weird chunk {{ His continued comfort }} ...fixed... [[ continued comfort ]]
	...found weird chunk {{ the emergency responders }} ...fixed... [[ emergency responders ]]
	...found weird chunk {{ a moment }} ...fixed... [[ moment ]]
	...found weird chunk {{ many others }} ...fixed... [[ others ]]
	...found weird chunk {{ what tomorrow }} ...fixed... [[ tomorrow ]]
	...found weird chunk {{ the resiliency }} ...fixed... [[ resiliency

	...found weird chunk {{ your gracious welcome }} ...fixed... [[ gracious welcome ]]
	...found weird chunk {{ just seven weeks }} ...fixed... [[ seven weeks ]]
	...found weird chunk {{ the oath }} ...fixed... [[ oath ]]
	...found weird chunk {{ the twelfth Governor }} ...fixed... [[ twelfth Governor ]]
	...found weird chunk {{ the history }} ...fixed... [[ history ]]
	...found weird chunk {{ our young state }} ...fixed... [[ young state ]]
	...found weird chunk {{ the People }} ...fixed... [[ People ]]
	...found weird chunk {{ their faith }} ...fixed... [[ faith ]]
	...found weird chunk {{ my vision }} ...fixed... [[ vision ]]
	...found weird chunk {{ our great state }} ...fixed... [[ great state ]]
	...found weird chunk {{ a simple truth }} ...fixed... [[ simple truth ]]
	...found weird chunk {{ its authority }} ...fixed... [[ authority ]]
	...found weird chunk {{ the people }} ...fixed... [[ people ]]
	...found weird chunk {{ The Court system }} ...fixed... [[ Court system ]]
	...fou

	...found weird chunk {{ the Legislature }} ...fixed... [[ Legislature ]]
	...found weird chunk {{ a lot }} ...fixed... [[ lot ]]
	...found weird chunk {{ our business }} ...fixed... [[ business ]]
	...found weird chunk {{ that’s }} ...fixed... [[ ’s ]]
	...found weird chunk {{ a really good thing }} ...fixed... [[ really good thing ]]
	...found weird chunk {{ the months }} ...fixed... [[ months ]]
	...found weird chunk {{ our promises }} ...fixed... [[ promises ]]
	...found weird chunk {{ the state }} ...fixed... [[ state ]]
	...found weird chunk {{ the first time }} ...fixed... [[ first time ]]
	...found weird chunk {{ the other public servants }} ...fixed... [[ other public servants ]]
	...found weird chunk {{ the oath }} ...fixed... [[ oath ]]
	...found weird chunk {{ a difference }} ...fixed... [[ difference ]]
	...found weird chunk {{ some fresh ideas }} ...fixed... [[ fresh ideas ]]
	...found weird chunk {{ the same reasons }} ...fixed... [[ same reasons ]]
	...found weird chunk

	...found weird chunk {{ the General Assembly }} ...fixed... [[ General Assembly ]]
	...found weird chunk {{ an occasion }} ...fixed... [[ occasion ]]
	...found weird chunk {{ a time }} ...fixed... [[ time ]]
	...found weird chunk {{ a determined course }} ...fixed... [[ determined course ]]
	...found weird chunk {{ the future }} ...fixed... [[ future ]]
	...found weird chunk {{ the celebration }} ...fixed... [[ celebration ]]
	...found weird chunk {{ my family }} ...fixed... [[ family ]]
	...found weird chunk {{ My wife }} ...fixed... [[ wife ]]
	...found weird chunk {{ my son }} ...fixed... [[ son ]]
	...found weird chunk {{ my daughter }} ...fixed... [[ daughter ]]
	...found weird chunk {{ a part }} ...fixed... [[ part ]]
	...found weird chunk {{ four grandchildren }} ...fixed... [[ grandchildren ]]
	...found weird chunk {{ a leave }} ...fixed... [[ leave ]]
	...found weird chunk {{ a special privilege }} ...fixed... [[ special privilege ]]
	...found weird chunk {{ my family }} ...f

	...found weird chunk {{ a champion }} ...fixed... [[ champion ]]
	...found weird chunk {{ your house }} ...fixed... [[ house ]]
	...found weird chunk {{ your commitment }} ...fixed... [[ commitment ]]
	...found weird chunk {{ our first month }} ...fixed... [[ first month ]]
	...found weird chunk {{ the honor }} ...fixed... [[ honor ]]
	...found weird chunk {{ the first time }} ...fixed... [[ first time ]]
	...found weird chunk {{ this chamber }} ...fixed... [[ chamber ]]
	...found weird chunk {{ that very kind and short introduction }} ...fixed... [[ very kind and short introduction ]]
	...found weird chunk {{ all the constitutional officers }} ...fixed... [[ the constitutional officers ]]
	...found weird chunk {{ your service }} ...fixed... [[ service ]]
	...found weird chunk {{ our state }} ...fixed... [[ state ]]
	...found weird chunk {{ our son }} ...fixed... [[ son ]]
	...found weird chunk {{ our lesson }} ...fixed... [[ lesson ]]
	...found weird chunk {{ the inauguration }} ...f

...

	...found weird chunk {{ My fellow Virginians }} ...fixed... [[ fellow Virginians ]]
	...found weird chunk {{ the Supreme Court }} ...fixed... [[ Supreme Court ]]
	...found weird chunk {{ the Virginia General Assembly— }} ...fixed... [[ Virginia General Assembly— ]]
	...found weird chunk {{ my wife }} ...fixed... [[ wife ]]
	...found weird chunk {{ First Lady Pamela Northam }} ...fixed... [[ Lady Pamela Northam ]]
	...found weird chunk {{ her focus }} ...fixed... [[ focus ]]
	...found weird chunk {{ every child }} ...fixed... [[ child ]]
	...found weird chunk {{ the Commonwealth }} ...fixed... [[ Commonwealth ]]
	...found weird chunk {{ a moment }} ...fixed... [[ moment ]]
	...found weird chunk {{ the first time }} ...fixed... [[ first time ]]
	...found weird chunk {{ the 400-year history }} ...fixed... [[ 400-year history ]]
	...found weird chunk {{ the Virginia General Assembly }} ...fixed... [[ Virginia General Assembly ]]
	...found weird chunk {{ a woman }} ...fixed... [[ woman ]]


	...found weird chunk {{ your inspiring words }} ...fixed... [[ inspiring words ]]
	...found weird chunk {{ the beautiful rendition }} ...fixed... [[ beautiful rendition ]]
	...found weird chunk {{ the national anthem }} ...fixed... [[ national anthem ]]
	...found weird chunk {{ your service }} ...fixed... [[ service ]]
	...found weird chunk {{ the Army National Guard }} ...fixed... [[ Army National Guard ]]
	...found weird chunk {{ a warm welcome }} ...fixed... [[ warm welcome ]]
	...found weird chunk {{ former Governor Gary Locke }} ...fixed... [[ Governor Gary Locke ]]
	...found weird chunk {{ his service }} ...fixed... [[ service ]]
	...found weird chunk {{ the new legislators }} ...fixed... [[ new legislators ]]
	...found weird chunk {{ your ranks }} ...fixed... [[ ranks ]]
	...found weird chunk {{ the people }} ...fixed... [[ people ]]
	...found weird chunk {{ this state }} ...fixed... [[ state ]]
	...found weird chunk {{ your families }} ...fixed... [[ families ]]
	...found weir

	...found weird chunk {{ a scene }} ...fixed... [[ scene ]]
	...found weird chunk {{ the practical jokers }} ...fixed... [[ practical jokers ]]
	...found weird chunk {{ a ball }} ...fixed... [[ ball ]]
	...found weird chunk {{ a ball }} ...fixed... [[ ball ]]
	...found weird chunk {{ an impact }} ...fixed... [[ impact ]]
	...found weird chunk {{ the first time }} ...fixed... [[ first time ]]
	...found weird chunk {{ the same amount }} ...fixed... [[ same amount ]]
	...found weird chunk {{ a lot }} ...fixed... [[ lot ]]
	...found weird chunk {{ your families }} ...fixed... [[ families ]]
	...found weird chunk {{ your businesses }} ...fixed... [[ businesses ]]
	...found weird chunk {{ And 



mine }} ...fixed... [[  



mine ]]
	...found weird chunk {{ our First Lady }} ...fixed... [[ First Lady ]]
	...found weird chunk {{ my daughter }} ...fixed... [[ daughter ]]
	...found weird chunk {{ a beautiful, 

beautifulest grand -- son }} ...fixed... [[ beautiful, 

beautifulest grand -- son ]]

	...found weird chunk {{ the members }} ...fixed... [[ members ]]
	...found weird chunk {{ the Wisconsin National Guard }} ...fixed... [[ Wisconsin National Guard ]]
	...found weird chunk {{ our armed forces }} ...fixed... [[ armed forces ]]
	...found weird chunk {{ a Wisconsin institution }} ...fixed... [[ Wisconsin institution ]]
	...found weird chunk {{ both the soul }} ...fixed... [[ the soul ]]
	...found weird chunk {{ our campus }} ...fixed... [[ campus ]]
	...found weird chunk {{ the spirit }} ...fixed... [[ spirit ]]
	...found weird chunk {{ our state }} ...fixed... [[ state ]]
	...found weird chunk {{ This year }} ...fixed... [[ year ]]
	...found weird chunk {{ the end }} ...fixed... [[ end ]]
	...found weird chunk {{ this year }} ...fixed... [[ year ]]
	...found weird chunk {{ the “stop }} ...fixed... [[ “stop ]]
	...found weird chunk {{ the top }} ...fixed... [[ top ]]
	...found weird chunk {{ the 5th Quarter post-game routines }} ...fixed... [[ 5th Quarter post-game routine

	...found weird chunk {{ the 65th Legislature }} ...fixed... [[ 65th Legislature ]]
	...found weird chunk {{ the Capitol }} ...fixed... [[ Capitol ]]
	...found weird chunk {{ What an
amazing team }} ...fixed... [[ an
amazing team ]]
	...found weird chunk {{ what an amazing group }} ...fixed... [[ an amazing group ]]
	...found weird chunk {{ the first time }} ...fixed... [[ first time ]]
	...found weird chunk {{ our state }} ...fixed... [[ state ]]
	...found weird chunk {{ a State }} ...fixed... [[ State ]]
	...found weird chunk {{ the Judiciary }} ...fixed... [[ Judiciary ]]
	...found weird chunk {{ a majority female bench }} ...fixed... [[ majority female bench ]]
	...found weird chunk {{ the monumental achievement }} ...fixed... [[ monumental achievement ]]
	...found weird chunk {{ the previous year }} ...fixed... [[ previous year ]]
	...found weird chunk {{ the first female Justice }} ...fixed... [[ first female Justice ]]
	...found weird chunk {{ the Peace }} ...fixed... [[ Peace ]

In [11]:
print(speech_noun_map['Alabama_SOTS.txt'][:50])

['Lieutenant Governor Ainsworth', 'Pro Tempore Marsh', 'Speaker McCutcheon', 'Speaker Pro Tempore Gaston', 'members', 'Alabama Legislature', 'Chief Justice Parker', 'justices', 'Alabama Supreme Court', 'fellow Alabamians', 'Mother Nature', '’s', 'form', 'state', 'significant devastation', 'least 23 innocent lives', 'Young children', 'life', 'Mothers', 'Fathers', 'Friends', 'neighbors', 'times', 'good Lord', 'continued comfort', 'healing hands', 'special thanks', 'emergency responders', 'local law enforcement', 'moment', 'silence', 'others', 'uncertainty', 'tomorrow', 'absolute certainty', 'resiliency', 'people', 'Alabama', 'time', 'entire nation', 'good people', 'Lee County', 'feet', '200 years', 'statehood', 'men', 'women', 'Alabama', 'nation', 'country']


In [24]:
def query_babelnet(chunk):
    """
    given a chunk of words, do call to BabelNet API and get results
    """
    params = {
        'lemma' : chunk,
        'searchLang' : 'EN',
        'key'  : BABEL_API_KEY
    }
    
    url = SERVICE_URL + '?' + urlencode(params) # format the request url
    request = Request(url)
    request.add_header('Accept-encoding', 'gzip') # ask for gzip response
    response = urlopen(request)
    
    if response.info().get('Content-Encoding') == 'gzip':
        buf = response.read()
        fname = chunk + "_" + str(datetime.date.today()) + ".gzip"
        save_babelnet_bytes_results(buf, fname)
        return json.loads(gzip.decompress(buf))
    else:
        raise Exception("unexpected response from BabelNet API")

def save_babelnet_bytes_results(babel_res, filename):
    """
    given BabelNet result, save to file
    """
    filename = os.path.join(BABEL_RESPONSE_DIR, filename)
    try:
        with open( filename, 'ab' ) as f: # append
            f.write(babel_res)
            f.close()
    except:
        with open( filename, 'wb' ) as f: # write/create
            f.write(babel_res)
            f.close()

In [25]:
babel_data = {}
chunk = speech_noun_map['Alabama_SOTS.txt'][8]
print(chunk)

Alabama Supreme Court


In [26]:
json_response = query_babelnet(chunk)
babel_data = json_response
print(babel_data)

[{'id': 'bn:14875491n', 'pos': 'NOUN', 'source': 'BABELNET'}]
